# Pre-Equilibrium Maximazation of Yield #

In [1]:
# make sure jupyter path is correct for loading local moudules
import sys
# path to steric_simulator module relative to notebook
sys.path.append("../../../")
import copy

In [2]:
from steric_free_simulator import ReactionNetwork, VectorizedRxnNet, VecSim, Optimizer, EquilibriumSolver
import networkx as nx
import torch
from torch import DoubleTensor as Tensor
import numpy as np

EnergyExplorer Module is not available. Check Rosetta installation. <ipykernel.iostream.OutStream object at 0x155550ac1080>


We'll start with the AP2 complex that we've worked with before. Pairwise $\Delta Gs$ were derived from the PDB structures via Rossetta

In [3]:
base_input = '../../input_files/tetramer_chaperone.pwr'
rn = ReactionNetwork(base_input, one_step=True)
rn.resolve_tree()

['default_assoc', 1.0]
['monomer_add_only', -1.0]
['chaperone', True]
['A']
100.0
['M']
100.0
['B']
100.0
['S']
100.0
['X']
4.05
Parsing rule...
SPLIT_01:  ['A(a)+B(b)', 'A(a!1).B(a!1)']
['A', 'B', '']
GGGGGGGGGgg
Parsing rule...
SPLIT_01:  ['A(b)+M(a)', 'A(b!1).M(a!1)']
['A', 'M', '']
GGGGGGGGGgg
Parsing rule...
SPLIT_01:  ['A(c)+S(a)', 'A(c!1).S(a!1)']
['A', 'S', '']
GGGGGGGGGgg
Parsing rule...
SPLIT_01:  ['B(b)+M(b)', 'B(b!1).M(b!1)']
['B', 'M', '']
GGGGGGGGGgg
Parsing rule...
SPLIT_01:  ['B(c)+S(b)', 'M(c!1).S(b!1)']
['B', 'S', '']
GGGGGGGGGgg
Parsing rule...
SPLIT_01:  ['M(c)+S(b)', 'M(c!1).S(b!1)']
['M', 'S', '']
GGGGGGGGGgg
Parsing rule...
SPLIT_01:  ['A(a!1).B(a!1).M(b!1)+X(a)', 'A(a)+B(b)+X(a)+M(b)']
['A(a!1).B(a!1).M(b!1)', 'X(a)']
GGGGGGGGGgg
Node-1 :  (0, {'struct': <networkx.classes.graph.Graph object at 0x1555500555c0>, 'copies': tensor([100.], dtype=torch.float64), 'subunits': 1})
Node-2 :  (0, {'struct': <networkx.classes.graph.Graph object at 0x1555500555c0>, 'copies':

Node-2 :  (8, {'struct': <networkx.classes.graph.Graph object at 0x1554bb535550>, 'copies': tensor([0.], dtype=torch.float64), 'subunits': 2})
-----
{'A', 'B'}
{'B', 'M'}
{'A'}
Steric hindrance detected
Node-1 :  (6, {'struct': <networkx.classes.graph.Graph object at 0x1554bb535208>, 'copies': tensor([0.], dtype=torch.float64), 'subunits': 2})
Node-2 :  (9, {'struct': <networkx.classes.graph.Graph object at 0x1554bb535978>, 'copies': tensor([0.], dtype=torch.float64), 'subunits': 2})
-----
{'A', 'B'}
{'S', 'M'}
{'A', 'B'}
False
Orig edges:  [('A', 'B')]
Nextn edges:  [('M', 'S')]
Item edges:  [('A', 'B'), ('M', 'S')]
Allowed edges: 
('A', 'B')
[True, True]
True
Allowed edges: 
('A', 'M')
[True, True]
False
############################3
Allowed edges: 
('A', 'S')
[True, True]
False
############################3
Allowed edges: 
('B', 'M')
[True, True]
False
############################3
Allowed edges: 
('B', 'S')
[True, True]
False
############################3
Allowed edges: 
('M', 'S')

Node-1 :  (16, {'struct': <networkx.classes.graph.Graph object at 0x1554bb4e7160>, 'copies': tensor([0.], dtype=torch.float64), 'subunits': 0})
Node-2 :  (0, {'struct': <networkx.classes.graph.Graph object at 0x1555500555c0>, 'copies': tensor([100.], dtype=torch.float64), 'subunits': 1})
-----
{'A', 'X', 'B', 'M'}
{'A'}
{'X', 'B', 'M'}
Steric hindrance detected
Node-1 :  (16, {'struct': <networkx.classes.graph.Graph object at 0x1554bb4e7160>, 'copies': tensor([0.], dtype=torch.float64), 'subunits': 0})
Node-2 :  (1, {'struct': <networkx.classes.graph.Graph object at 0x1554bb9849b0>, 'copies': tensor([100.], dtype=torch.float64), 'subunits': 1})
-----
{'A', 'X', 'B', 'M'}
{'M'}
{'A', 'X', 'B'}
Steric hindrance detected
Node-1 :  (16, {'struct': <networkx.classes.graph.Graph object at 0x1554bb4e7160>, 'copies': tensor([0.], dtype=torch.float64), 'subunits': 0})
Node-2 :  (2, {'struct': <networkx.classes.graph.Graph object at 0x155550064fd0>, 'copies': tensor([100.], dtype=torch.float64),

In [4]:
print(rn.rxn_class)
print(rn.chap_uid_map)

{(1, 1): [0, 1, 2, 3, 4, 7], (2, 1): [5, 6, 8, 9, 10, 12, 13, 14, 20, 21, 22, 23], (3, 1): [11, 15, 16, 24, 25], (2, 2): [17, 18, 19]}
{4: [26, 16]}


In [5]:
uid_dict = {}
sys.path.append("../../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    print(n,"--",gtostr(rn.network.nodes[n]['struct']))
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_dict[(n,k)] = uid

print(uid_dict)

0 -- A
1 -- M
2 -- B
3 -- S
4 -- X
5 -- AM
6 -- AB
7 -- AS
8 -- BM
9 -- MS
10 -- ABM
11 -- AMS
12 -- BS
13 -- ABS
14 -- BMS
15 -- ABMS
16 -- ABMX
{(0, 5): 0, (0, 6): 1, (0, 7): 2, (0, 10): 20, (0, 11): 21, (0, 13): 22, (0, 15): 25, (1, 5): 0, (1, 8): 3, (1, 9): 4, (1, 10): 5, (1, 11): 6, (1, 14): 23, (1, 15): 24, (2, 6): 1, (2, 8): 3, (2, 12): 7, (2, 10): 8, (2, 13): 9, (2, 14): 10, (2, 15): 11, (3, 7): 2, (3, 9): 4, (3, 12): 7, (3, 11): 12, (3, 13): 13, (3, 14): 14, (3, 15): 15, (4, 16): 16, (5, 10): 8, (5, 11): 12, (5, 15): 17, (6, 10): 5, (6, 13): 13, (6, 15): 18, (7, 11): 6, (7, 13): 9, (7, 15): 19, (8, 14): 14, (8, 15): 19, (8, 10): 20, (9, 14): 10, (9, 15): 18, (9, 11): 21, (10, 15): 15, (10, 16): 16, (11, 15): 11, (12, 15): 17, (12, 13): 22, (12, 14): 23, (13, 15): 24, (14, 15): 25, (16, 0): 26, (16, 1): 26, (16, 2): 26, (16, 4): 26}


In [6]:
#Do modifications here
#Changing Initial Conditions
import networkx as nx
#Changin k_on
new_kon = torch.zeros([rn._rxn_count], requires_grad=True).double()
new_kon = new_kon + Tensor([1.]*np.array(1e0))
new_kon[16] = 6.4
new_kon[26] = 10
update_kon_dict = {}
for edge in rn.network.edges:
    print(rn.network.get_edge_data(edge[0],edge[1]))
    update_kon_dict[edge] = new_kon[uid_dict[edge]]

nx.set_edge_attributes(rn.network,update_kon_dict,'k_on')

# for edge in rn.network.edges:
#     print(rn.network.get_edge_data(edge[0],edge[1]))
vec_rn = VectorizedRxnNet(rn, dev='cpu',assoc_is_param=False,chap_is_param=True)
print(vec_rn.kon)

#Changing initial concentrations

{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 0}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 1}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 2}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-40.], dtype=torch.float64), 'uid': 20}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-40.], dtype=torch.float64), 'uid': 21}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-40.], dtype=torch.float64), 'uid': 22}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-60.], dtype=torch.float64), 'uid': 25}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 0}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 3}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 4}
{'k_on

## The Equilibrium Solution ##
First we will find the equilibrium solution for this system.

In [7]:
# vec_rn.update_reaction_net(rn)
# poly_system = EquilibriumSolver(rn)
# solution = poly_system.solve()
# print(solution)
# if solution == None:
#     print("No Equilibrium solution")
# else:
#     print(solution)
#     print("Equilibrium expected yield: ", 100 * solution[-1] / min(vec_rn.initial_copies[:vec_rn.num_monomers]), '%')
# print(vec_rn.kon)

In [8]:
# uid_dict = {}
# sys.path.append("../")
# import numpy as np
# from reaction_network import gtostr
# from torch import DoubleTensor as Tensor

# def get_max_edge(n):
#     """
#     Calculates the max rate (k_on) for a given node
#     To find out the maximum flow path to the final complex starting from the current node.
    
#     Can also calculate the total rate of consumption of a node by summing up all rates. 
#     Can tell which component is used quickly.
#     """
#     try:
#         edges = rn.network.out_edges(n)
#         #Loop over all edges
#         #Get attributes
#         if len(edges)==0:
#             return(False)
#         kon_max = -1
#         next_node = -1
        
#         kon_sum = 0
#         for edge in edges:
#             data = rn.network.get_edge_data(edge[0],edge[1])
#             #print(data)
#             #Get uid
#             uid = data['uid']
#             #Get updated kon
#             temp_kon = vec_rn.kon[uid]
#             kon_sum+=temp_kon
            
# #             #Calculate k_off also
# #             std_c = Tensor([1.])
# #             l_kon = torch.log(temp_kon)
# #             l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (self._R * self._T)) + l_kon + torch.log(std_c)
#             if temp_kon > kon_max:
#                 kon_max = temp_kon
#                 next_node=edge[1]
#         return(kon_max,next_node,kon_sum)
#     except Exception as err:
#         raise(err)

# pathway = []
# kon_sumarray = []
# total_con_rate = {}
# for n in rn.network.nodes():
    
#     n_str = gtostr(rn.network.nodes[n]['struct']) 
    
#     paths = [n_str]
#     kon_sum = 0
#     temp_node = n
#     max_edge = True
#     consumption_rate = 0
#     if n < len(rn.network.nodes()):#num_monomers:
# #         print("Current node: ")
# #         print(n_str)
#         while max_edge:
#             max_edge = get_max_edge(temp_node)
#             if max_edge:
#                 total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
#                 temp_node = max_edge[1]
#                 kon_sum += max_edge[0].item()
                
# #                 print("Next node: ")
# #                 print(temp_node)

#                 paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
#             else:
#                 break
#         pathway.append(paths)
#         kon_sumarray.append(kon_sum)
#         paths=[]

# print(pathway)
# print(kon_sumarray)
# #print(total_con_rate)

In [9]:
# if solution != None:
#     for k,v in sorted(total_con_rate.items(),key=lambda x : x[1]):
#         print(k," : ", v.item())

In [10]:
# if solution !=None:
#     for k,v in sorted(net_flux.items(),key=lambda x : x[1]):
#         print(k," : ", v)

## Using the optimizer with a 1 second simulation runtime ##

In [11]:
runtime=10
vec_rn.reset(reset_params=True)
optim = Optimizer(reaction_network=vec_rn,
                  sim_runtime=runtime,
                  optim_iterations=100,
                  learning_rate=[2e-3,1e-8,2e-3],
                  device='cpu',method="RMSprop",mom=0.5)
optim.rn.update_reaction_net(rn)
optim.optimize(conc_scale=1e-1,mod_factor=1.0,conc_thresh=1e-1,mod_bool=True,yield_species=15,max_yield=0,chap_mode=3)

Using CPU
Reaction Parameters before optimization: 
[Parameter containing:
tensor(4.0500, dtype=torch.float64, requires_grad=True), Parameter containing:
tensor(10., dtype=torch.float64, requires_grad=True), Parameter containing:
tensor(6.4000, dtype=torch.float64, requires_grad=True)]
Optimizer State: <bound method Optimizer.state_dict of RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-08
    lr: 0.0081
    momentum: 0.5
    weight_decay: 0

Parameter Group 1
    alpha: 0.99
    centered: False
    eps: 1e-08
    lr: 1e-07
    momentum: 0.5
    weight_decay: 0

Parameter Group 2
    alpha: 0.99
    centered: False
    eps: 1e-08
    lr: 0.0128
    momentum: 0.5
    weight_decay: 0
)>
Using CPU
Start of simulation: memory Used:  14.6
Next time:  tensor(10.1191, dtype=torch.float64, grad_fn=<AddBackward0>)
Max Possible Yield:  tensor(100., dtype=torch.float64, grad_fn=<MinBackward1>)
yield on sim iteration 0 was 96.8%
current params: [tensor(4.0500, dtype=tor

Grad:  tensor(0.0242, dtype=torch.float64)Grad:  tensor(0.0067, dtype=torch.float64)Grad:  tensor(0.0046, dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  15.6
Next time:  tensor(10.1767, dtype=torch.float64, grad_fn=<AddBackward0>)
Max Possible Yield:  tensor(100., dtype=torch.float64, grad_fn=<MinBackward1>)
yield on sim iteration 11 was 97.0%
current params: [tensor(3.2039, dtype=torch.float64), tensor(10.0000, dtype=torch.float64), tensor(5.0456, dtype=torch.float64)]
Penalty:  tensor(0., dtype=torch.float64, grad_fn=<AddBackward0>) Dimer yield:  tensor(4.4973e-05, dtype=torch.float64, grad_fn=<DivBackward0>) ABT yield:  tensor(7.3530e-05, dtype=torch.float64, grad_fn=<DivBackward0>)
Grad:  tensor(0.0245, dtype=torch.float64)Grad:  tensor(0.0067, dtype=torch.float64)Grad:  tensor(0.0047, dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  15.6
Next time:  tensor(10.1878, dtype=torch.float64, grad_fn=<AddBackward0>)
Max Possible Yield:  tensor(100., dt

Grad:  tensor(0.0285, dtype=torch.float64)Grad:  tensor(0.0062, dtype=torch.float64)Grad:  tensor(0.0060, dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  15.3
Next time:  tensor(10.0241, dtype=torch.float64, grad_fn=<AddBackward0>)
Max Possible Yield:  tensor(100., dtype=torch.float64, grad_fn=<MinBackward1>)
yield on sim iteration 23 was 97.1%
current params: [tensor(2.6374, dtype=torch.float64), tensor(10.0000, dtype=torch.float64), tensor(4.1053, dtype=torch.float64)]
Penalty:  tensor(0., dtype=torch.float64, grad_fn=<AddBackward0>) Dimer yield:  tensor(6.6354e-05, dtype=torch.float64, grad_fn=<DivBackward0>) ABT yield:  tensor(7.2714e-05, dtype=torch.float64, grad_fn=<DivBackward0>)
Grad:  tensor(0.0289, dtype=torch.float64)Grad:  tensor(0.0061, dtype=torch.float64)Grad:  tensor(0.0061, dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  15.4
Next time:  tensor(10.1986, dtype=torch.float64, grad_fn=<AddBackward0>)
Max Possible Yield:  tensor(100., dt

Grad:  tensor(0.0332, dtype=torch.float64)Grad:  tensor(0.0055, dtype=torch.float64)Grad:  tensor(0.0078, dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  16.1
Next time:  tensor(10.0209, dtype=torch.float64, grad_fn=<AddBackward0>)
Max Possible Yield:  tensor(100., dtype=torch.float64, grad_fn=<MinBackward1>)
yield on sim iteration 35 was 97.3%
current params: [tensor(2.1752, dtype=torch.float64), tensor(10.0000, dtype=torch.float64), tensor(3.3067, dtype=torch.float64)]
Penalty:  tensor(0., dtype=torch.float64, grad_fn=<AddBackward0>) Dimer yield:  tensor(9.8011e-05, dtype=torch.float64, grad_fn=<DivBackward0>) ABT yield:  tensor(7.1393e-05, dtype=torch.float64, grad_fn=<DivBackward0>)
Grad:  tensor(0.0336, dtype=torch.float64)Grad:  tensor(0.0055, dtype=torch.float64)Grad:  tensor(0.0080, dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  16.2
Next time:  tensor(10.0030, dtype=torch.float64, grad_fn=<AddBackward0>)
Max Possible Yield:  tensor(100., dt

Grad:  tensor(0.0388, dtype=torch.float64)Grad:  tensor(0.0048, dtype=torch.float64)Grad:  tensor(0.0108, dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  16.9
Next time:  tensor(10.2414, dtype=torch.float64, grad_fn=<AddBackward0>)
Max Possible Yield:  tensor(100., dtype=torch.float64, grad_fn=<MinBackward1>)
yield on sim iteration 47 was 97.6%
current params: [tensor(1.7559, dtype=torch.float64), tensor(10.0000, dtype=torch.float64), tensor(2.5433, dtype=torch.float64)]
Penalty:  tensor(0., dtype=torch.float64, grad_fn=<AddBackward0>) Dimer yield:  tensor(0.0002, dtype=torch.float64, grad_fn=<DivBackward0>) ABT yield:  tensor(6.9791e-05, dtype=torch.float64, grad_fn=<DivBackward0>)
Grad:  tensor(0.0393, dtype=torch.float64)Grad:  tensor(0.0047, dtype=torch.float64)Grad:  tensor(0.0111, dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  17.0
Next time:  tensor(10.1645, dtype=torch.float64, grad_fn=<AddBackward0>)
Max Possible Yield:  tensor(100., dtype=

Grad:  tensor(0.0456, dtype=torch.float64)Grad:  tensor(0.0039, dtype=torch.float64)Grad:  tensor(0.0165, dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  17.7
Next time:  tensor(10.0455, dtype=torch.float64, grad_fn=<AddBackward0>)
Max Possible Yield:  tensor(100., dtype=torch.float64, grad_fn=<MinBackward1>)
yield on sim iteration 59 was 97.7%
current params: [tensor(1.3569, dtype=torch.float64), tensor(10.0000, dtype=torch.float64), tensor(1.7556, dtype=torch.float64)]
Penalty:  tensor(0., dtype=torch.float64, grad_fn=<AddBackward0>) Dimer yield:  tensor(0.0003, dtype=torch.float64, grad_fn=<DivBackward0>) ABT yield:  tensor(8.4106e-05, dtype=torch.float64, grad_fn=<DivBackward0>)
Grad:  tensor(0.0463, dtype=torch.float64)Grad:  tensor(0.0038, dtype=torch.float64)Grad:  tensor(0.0173, dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  17.7
Next time:  tensor(10.1902, dtype=torch.float64, grad_fn=<AddBackward0>)
Max Possible Yield:  tensor(100., dtype=

Grad:  tensor(0.0526, dtype=torch.float64)Grad:  tensor(0.0024, dtype=torch.float64)Grad:  tensor(0.0344, dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  18.4
Next time:  tensor(10.0177, dtype=torch.float64, grad_fn=<AddBackward0>)
Max Possible Yield:  tensor(100., dtype=torch.float64, grad_fn=<MinBackward1>)
yield on sim iteration 71 was 97.1%
current params: [tensor(0.9698, dtype=torch.float64), tensor(10.0000, dtype=torch.float64), tensor(0.8545, dtype=torch.float64)]
Penalty:  tensor(3.0173, dtype=torch.float64, grad_fn=<AddBackward0>) Dimer yield:  tensor(0.0024, dtype=torch.float64, grad_fn=<DivBackward0>) ABT yield:  tensor(0.0002, dtype=torch.float64, grad_fn=<DivBackward0>)
Grad:  tensor(0.0530, dtype=torch.float64)Grad:  tensor(0.0022, dtype=torch.float64)Grad:  tensor(0.0379, dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  18.5
Next time:  tensor(10.0727, dtype=torch.float64, grad_fn=<AddBackward0>)
Max Possible Yield:  tensor(100., dtype=

Max Possible Yield:  tensor(100., dtype=torch.float64, grad_fn=<MinBackward1>)
yield on sim iteration 96 was 0.0%
current params: [tensor(nan, dtype=torch.float64), tensor(nan, dtype=torch.float64), tensor(nan, dtype=torch.float64)]
Penalty:  tensor(nan, dtype=torch.float64, grad_fn=<AddBackward0>) Dimer yield:  tensor(0., dtype=torch.float64, grad_fn=<DivBackward0>) ABT yield:  tensor(0., dtype=torch.float64, grad_fn=<DivBackward0>)
Grad:  tensor(nan, dtype=torch.float64)Grad:  tensor(nan, dtype=torch.float64)Grad:  tensor(nan, dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  18.9
Max Possible Yield:  tensor(100., dtype=torch.float64, grad_fn=<MinBackward1>)
yield on sim iteration 97 was 0.0%
current params: [tensor(nan, dtype=torch.float64), tensor(nan, dtype=torch.float64), tensor(nan, dtype=torch.float64)]
Penalty:  tensor(nan, dtype=torch.float64, grad_fn=<AddBackward0>) Dimer yield:  tensor(0., dtype=torch.float64, grad_fn=<DivBackward0>) ABT yield:  tensor(0., d

In [12]:
# optim.optimize()

In [13]:
# optim.optimize()

In [14]:
# optim.optimize()

In [15]:
def calc_var(v1,v2):
    sq_sum=0
    for i in range(len(v1)):
        sq_sum=(v1[i]-v2[i])**2+sq_sum
    
    sq_sum = ((sq_sum)**0.5)/(len(v1)-1)
    return(sq_sum)

In [16]:
yields= []
ab_yields=[]
abt_yields=[]
final_copies=[]
final_rates = []
final_t50 = []
final_t85 = []
final_t95 = []
final_t99 = []
times = []
n_copies=len(rn.chap_uid_map.keys())
n_rates = 2*n_copies
for i in range(len(optim.final_yields)):
    yields.append(optim.final_yields[i][0].item())
    ab_yields.append(optim.dimer_max[i].item())
    abt_yields.append(optim.chap_max[i].item())
    times.append(optim.endtimes[i])
    rate_params = []
    copy_params = []
    for j in range(n_copies):
#     print(optim.final_solns[i])
        copy_params.append(optim.final_solns[i][j].numpy())
    for r in range(n_rates):
        rate_params.append(optim.final_solns[i][r+n_copies].numpy())
    
    final_copies.append(copy_params)
    final_rates.append(rate_params)
    
    if type(optim.final_t50[i])==int:
        final_t50.append(1) 
    else:
        final_t50.append(optim.final_t50[i].item()) 
    if type(optim.final_t85[i])==int:
        final_t85.append(1) 
    else:
        final_t85.append(optim.final_t85[i].item()) 
    if type(optim.final_t95[i])==int:
        final_t95.append(1)
    else:
        final_t95.append(optim.final_t95[i].item())
    if type(optim.final_t99[i])==int:
        final_t99.append(1)
    else:
        final_t99.append(optim.final_t99[i].item())

sort_indx=np.argsort(np.array(yields))


# unsorted_yields=np.array(yields)
# unsorted_copies = np.array(final_copies)
# unsorted_rates = np.array(final_rates)

# mask = unsorted_yields>0.2

sorted_yields=np.array(yields)#[sort_indx]
sorted_ab_yields=np.array(ab_yields)
sorted_abt_yields=np.array(abt_yields)
sorted_times = np.array(times)
sorted_copies = np.array(final_copies)#[sort_indx]
sorted_rates = np.array(final_rates)#[sort_indx]
sorted_t50 = np.array(final_t50)
sorted_t85 = np.array(final_t85)
sorted_t95 = np.array(final_t95)
sorted_t99 = np.array(final_t99)

print("Parameters with Max yield: ")
print("Max Yield: ",sorted_yields[-1],"\nParams: ",sorted_copies[-1], sorted_rates[-1])
print("Iteration: ", sort_indx[-1])

Parameters with Max yield: 
Max Yield:  0.5989112369170285 
Params:  [0.74211134] [9.99997067 0.01662728]
Iteration:  60


In [17]:
print(len(optim.final_yielssds))

AttributeError: 'Optimizer' object has no attribute 'final_yielssds'

In [ ]:


# # Writing all solutions to a file
# filename="Tetramer_Solutions_Chaperone_20kT_Mode2"
# with open(filename,'a') as fl:
#     fl.write("#! Yield\tAB_Yield\tABT_Yield\tTimes\tC_x\tk1\tk2\tt50\tt85\tt90\n")
#     for i in range(1,len(sorted_yields)):
#         fl.write("%f" %(sorted_yields[i]))
#         fl.write("\t%f" %(sorted_ab_yields[i]))
#         fl.write("\t%f" %(sorted_abt_yields[i]))
#         fl.write("\t%f" %(sorted_times[i]))
        
#         for j in range(sorted_copies[0].shape[0]):         
#             fl.write("\t%f" %(sorted_copies[i][j]))
#         for r1 in range(sorted_rates[0].shape[0]):
#             fl.write("\t%f" %(sorted_rates[i][r1]))
#         fl.write("\t%f\t%f\t%f\n" %(sorted_t50[i],sorted_t85[i],sorted_t95[i]))
                 

In [ ]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_val = {'reactants':reactants,'kon':v['k_on'],'score':v['rxn_score'],'koff':v['k_off']}
        if uid not in uid_dict.keys():
            uid_dict[uid] = uid_val
    print(gtostr(rn.network.nodes[n]['struct']))
    #for r_set in rn.get_reactant_sets(n):
    #    print(tuple(r_set))
    #print(rn.network[n]['struct'])
ind_sort = np.argsort(vec_rn.kon.detach().numpy())
for i in ind_sort:
    print(vec_rn.kon[i])
    print(uid_dict[i])

In [ ]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
from torch import DoubleTensor as Tensor

def get_max_edge(n):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        if len(edges)==0:
            return(False)
        kon_max = -1
        next_node = -1
        
        kon_sum = 0
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']
            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
#             #Calculate k_off also
#             std_c = Tensor([1.])
#             l_kon = torch.log(temp_kon)
#             l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (self._R * self._T)) + l_kon + torch.log(std_c)
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)

pathway = []
kon_sumarray = []
total_con_rate = {}
for n in rn.network.nodes():
    
    n_str = gtostr(rn.network.nodes[n]['struct']) 
    
    paths = [n_str]
    kon_sum = 0
    temp_node = n
    max_edge = True
    consumption_rate = 0
    if n < len(rn.network.nodes()):#num_monomers:
#         print("Current node: ")
#         print(n_str)
        while max_edge:
            max_edge = get_max_edge(temp_node)
            if max_edge:
                total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                temp_node = max_edge[1]
                kon_sum += max_edge[0].item()
                
#                 print("Next node: ")
#                 print(temp_node)

                paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
            else:
                break
        pathway.append(paths)
        kon_sumarray.append(kon_sum)
        paths=[]

print(pathway)
print(kon_sumarray)
#print(total_con_rate)

In [ ]:
for k,v in sorted(total_con_rate.items(),key=lambda x : x[1]):
    print(k," : ", v.item())

Let's first visualize some of the data.

**Without any optimization**


In [ ]:
nodes_list = ['A','B','S','M','AB','BMS','ABS','AMS','ABMS','AM','AS']
#nodes_list = ['A','B','ABMS']
optim.plot_observable(0,nodes_list)


**After 750 optimization iterations**


In [ ]:
optim.plot_observable(-1,nodes_list)


In [ ]:
optim.plot_yield()

It seems like we've found a stable solution that produces greater yield than equilibrium. This should be thermodynamically
impossible. Let's try to find an explanation. We'll run simulations using the learned optimal parameters at a few different
timescales.

In [ ]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots(1, 3)
optim_rn = optim.rn
for i, runtime in enumerate([1, 8, 64]):
    optim_rn.reset()
    sim = VecSim(optim_rn, runtime, device='cpu')
    y = sim.simulate()
    sim.plot_observable(nodes_list,ax=ax[i],)
    ax[i].set_title("runtime: " + str(runtime) + " seconds")
fig.set_size_inches(18, 6)
node_map = {}
for node in rn.network.nodes():
    node_map[gtostr(rn.network.nodes[node]['struct'])] = node

print(node_map)
plt.show()

In [ ]:
node_map = {}
for node in rn.network.nodes():
    node_map[gtostr(rn.network.nodes[node]['struct'])] = node

print(node_map)
def get_max_edge(n):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        kon_max = -1
        next_node = -1

        kon_sum = 0
        total_flux_outedges = 0
        total_flux_inedges = 0
        if len(edges)==0:
            return(False)
            
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
             
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)

        
def get_node_flux(n):
    total_flux_outedges = 0
    total_flux_inedges = 0
    #Go over all the out edges
    edges_out = rn.network.out_edges(n)
    if len(edges_out)>0:

        for edge in edges_out:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]

            #Calculate k_off also
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Getting conc. of reactants and products
            #Get product
            prod = gtostr(rn.network.nodes[edge[1]]['struct']) 
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))

            #Net flux from this edge = Generation - consumption
            edge_flux = koff*vec_rn.copies_vec[edge[1]] - temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])
            #edge_flux = koff*vec_rn.copies_vec[edge[1]] 

            print("Reaction: ", gtostr(rn.network.nodes[edge[0]]['struct']), "+",react," -> ",prod)
            print("Net flux: ",edge_flux)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Reaction data OUTWARD: ")
            print(data)

            total_flux_outedges+=edge_flux
    
    #Now go over all the in edges
    edges_in = rn.network.in_edges(n)
    react_list = []
    if len(edges_in) > 0:
        for edge in edges_in:
            if edge[0] in react_list:
                continue
            data = rn.network.get_edge_data(edge[0],edge[1])
            uid = data['uid']


            #Get generation rates; which would be kon
            temp_kon = vec_rn.kon[uid]

            #Get consumption rates; which is k_off
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Get conc. of reactants and products
            prod = gtostr(rn.network.nodes[edge[1]]['struct'])
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))
            react_list.append(node_map[react])
            #Net flux from this edge = Generation - consumption
            edge_flux_in = temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])- koff*vec_rn.copies_vec[edge[1]]
            #edge_flux_in = koff*vec_rn.copies_vec[edge[1]]
            


            print("Reaction: ", prod ," -> ",gtostr(rn.network.nodes[edge[0]]['struct']), "+",react)
            print("Net flux: ",edge_flux_in)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Raction data INWARD: ")
            print(data)

            total_flux_inedges+=edge_flux_in
    net_node_flux = total_flux_outedges + total_flux_inedges
    
    return(net_node_flux)
    
pathway = []
kon_sumarray = []
total_con_rate = {}
net_flux = {}
for n in rn.network.nodes():
    
    n_str = gtostr(rn.network.nodes[n]['struct']) 
    
    paths = [n_str]
    kon_sum = 0
    temp_node = n
    max_edge = True
    consumption_rate = 0
    if n < len(rn.network.nodes()):#num_monomers:
#         print("Current node: ")
#         print(n_str)
        while max_edge:
            max_edge = get_max_edge(temp_node)
            if max_edge:
                total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                
                temp_node = max_edge[1]
                kon_sum += max_edge[0].item()
                
                
#                 print("Next node: ")
#                 print(temp_node)

                paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
            else:
                break
        pathway.append(paths)
        kon_sumarray.append(kon_sum)
        paths=[]
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")
    print("|                                                                             |")
    node_flux = get_node_flux(n)
    net_flux[gtostr(rn.network.nodes[n]['struct'])] = node_flux
    print("|                                                                             |")
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")

print(pathway)
print(kon_sumarray)

#print(total_con_rate)

In [ ]:
for k,v in sorted(net_flux.items(),key=lambda x : x[1]):
    print(k," : ", v)

print(vec_rn.copies_vec)
print(vec_rn.kon)

In [ ]:
print(solution)
poly_system = EquilibriumSolver(rn)
solution = poly_system.solve(init_val=vec_rn.copies_vec.detach().numpy().tolist())
#solution = poly_system.solve(verifyBool = False)
if solution == None:
    print("No Equilibrium solution")
else:
    print(solution)
    print("Equilibrium expected yield: ", 100 * solution[-1] / min(vec_rn.initial_copies[:vec_rn.num_monomers]), '%')
print(vec_rn.kon)

Clearly, the equilibrium reached by the system still matches the equilibrium solution. We have however found a set of parameters that can increase available complete AP2 at some point before equilibrium to levels significantly higher than at equilibrium. We don't observe any trapping, but have uncovered an interesting effect. 

Now we'll move on to looking at ARP23. This is 7 subunits, which drastically increases the number of possible reactions. Expect longer runtimes. 